# Homework 7

## Due Thursday, November 29th 2018 at 11:59 PM.

### Be sure to push the final version of your notebook to your GitHub repo.  Follow the instructions on the course website.

### Topics
####  [Part 1](#part_1):  Database schema [15 points]
* [Problem 1](#p1.1). Schema [15 points]

####  [Part 2](#part_2):  Insert records [35 points]
* [Problem 2](#p2.1). Baseline model [15 points]
* [Problem 3](#p2.2). Reduced model [10 points]
* [Problem 4](#p2.3). L1 penalty model [10 points]

####  [Part 3](#part_3):  Queries [20 pts]
* [Problem 5](#p3.1). Best model coefficients [10 points]
* [Problem 6](#p3.2). Best model score [10 points]

---

<a id='part_1'></a>
# Part 1:  Database schema

<a id='p1.1'></a>
## Problem 1 (15 points): 

In this problem you will set up a SQL database using the `sqllite` package in Python. The purpose of the database will be to store parameters and model results related to a simple *Logistic Regression* problem. Rather than keeping the results in `Numpy` arrays as we usually do, the idea here is to make use of a `SQL` database to materialize the results so that it can easily be accessed from disk at a later stage.

The design of the database should be flexible enough so that the results from different model iterations can be stored in the database. It should also be able to deal with a different set of features by model iteration.

A list of the tables to include in the database and the relevant fields in each table is shown below (tables are in bold):

**model_params**: 
* id 
* desc 
* param_name
* value

**model_coeffs**
* id 
* desc 
* feature_name
* value

**model_results**
* id 
* desc 
* train_score
* test_score

Create a `SQL` database called `regression.sqlite` containing the three tables shown above.

## Problem 1 (SOLUTION)

In [1]:
import sqlite3

In [2]:
# creating a new database called regression
db = sqlite3.connect('regression.sqlite')
cursor = db.cursor()

# drop the tables if the tables already exist
cursor.execute("DROP TABLE IF EXISTS model_params")
cursor.execute("DROP TABLE IF EXISTS model_coeffs")
cursor.execute("DROP TABLE IF EXISTS model_results")

# creating the model_params table
cursor.execute('''CREATE TABLE model_params (
               id INTEGER NOT NULL, 
               desc TEXT, 
               param_name TEXT, 
               value BLOB)''')

# creating the model_coeffs table
cursor.execute('''CREATE TABLE model_coeffs (
               id INTEGER NOT NULL, 
               desc TEXT, 
               feature_name TEXT, 
               value REAL)''')

# creating the model_results table
cursor.execute('''CREATE TABLE model_results (
               id INTEGER NOT NULL, 
               desc TEXT, 
               train_score REAL, 
               test_score REAL)''')

db.commit() # Commit changes to the database

<a id='part_2'></a>
# Part 2: Insert records

In this section you will populate the database you created in the previous question with some records for a number of different model iterations / scenarios.

<a id='p2.1'></a>
## Problem 2 (15 points): 
Create a baseline Logistic Regression model using the provided code (below).  Insert the relevant arrays into the corresponding tables in the database.

**model_params**
Values from the [`get_params`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.get_params) method.

**model_coeffs**
Coefficient and intercept values of the fitted model (see `coef_` and `intercept_` attributes in the documentation).

**model_results**
Train and validation accuracy obtained from the [`score`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression.score) method.


#### Remarks
* Reference scikit-learn documentation to get more detail on the methods / attributes list above:  
[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

* Note that the *id* and *desc* are just identifier fields used to identify the results from a specific model iteration or scenario. For example for the baseline model you could set *id = 1* and *desc = "Baseline model"*.


#### Suggestions
You may want to create a function to save data to the database.  You will be able to re-use this function in subsequent sections.

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer

%matplotlib inline

In [4]:
# Load data
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=87)

In [5]:
# Fit model
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Problem 2 (SOLUTION)

In [6]:
def save_model_data(model_id, model_desc, feature_names, X_train, X_test, y_train, y_test, model):
    '''Function that stores the relevant model data into the Regression database.
    
    INPUT
    =========
    model_id       : int
                     id of model
    model_desc     : str
                     description of the model
    feature_names  : str list
                     list of names of the features used for model
    X_train        : 2D array or DataFrame
                     Matrix of parameter values that model fits on
    X_test         : 2D array or DataFrame
                     Matrix of parameter values that model predicts on
    y_train        : 1D array
                     Array of response variables that model fits on
    y_test         : 1D array
                     Array of response variables that model wishes to predict
    model          : sklearn.linear_model
                     the model
    
    
    OUTPUT
    =========
    Returns bTrue if saving data was successful.
    '''
    
    # storing parameter values in the model_params table
    params = model.get_params()
    for key in params:
        par_to_insert = (model_id, model_desc, key, params[key])
        cursor.execute('''INSERT INTO model_params 
                          (id, desc, param_name, value)
                          VALUES (?, ?, ?, ?)''', par_to_insert)
    
    # storing intercept and coefficient values into model_coeffs table
    intercept_to_insert = (model_id, model_desc, "intercept", model.intercept_[0])
    cursor.execute('''INSERT INTO model_coeffs
                      (id, desc, feature_name, value)
                      VALUES (?, ?, ?, ?)''', intercept_to_insert)
    
    # creating a dictionary of the coefficients for each feature
    coeffs = dict(zip(feature_names, model.coef_[0]))
    for key in coeffs:
        coeff_to_insert = (model_id, model_desc, key, coeffs[key])
        cursor.execute('''INSERT INTO model_coeffs 
                          (id, desc, feature_name, value)
                          VALUES (?, ?, ?, ?)''', coeff_to_insert)
        
    # storing the training and test scores into model_results table
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    result_to_insert = (model_id, model_desc, train_score, test_score)
    cursor.execute('''INSERT INTO model_results
                      (id, desc, train_score, test_score)
                      VALUES (?, ?, ?, ?)''', result_to_insert)  
    
    db.commit()
    
    # saving data was successful
    return True

**USING ```save_model_data``` TO STORE RESULTS**

In [7]:
save_model_data(model_id = 1, 
                model_desc = "Baseline model", 
                feature_names = data.feature_names, 
                X_train = X_train, 
                X_test = X_test, 
                y_train = y_train, 
                y_test = y_test, 
                model = clf)

True

---

**Method to visualize the tables (and making sure ```save_model_data``` works)**

In [8]:
model_params_cols = ['id', 'desc', 'param_name', 'value']
model_coeffs_cols = ['id', 'desc', 'feature_name', 'value']
model_results_cols = ['id', 'desc', 'train_score', 'test_score']

In [9]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [10]:
query = '''SELECT * FROM model_params'''
display(viz_tables(model_params_cols, query))

query = '''SELECT * FROM model_coeffs'''
display(viz_tables(model_coeffs_cols, query))

query = '''SELECT * FROM model_results'''
display(viz_tables(model_results_cols, query))

,id,desc,param_name,value
0,1,Baseline model,C,1
1,1,Baseline model,class_weight,None
2,1,Baseline model,dual,0
3,1,Baseline model,fit_intercept,1
4,1,Baseline model,intercept_scaling,1
5,1,Baseline model,max_iter,100
6,1,Baseline model,multi_class,ovr
7,1,Baseline model,n_jobs,1
8,1,Baseline model,penalty,l2
9,1,Baseline model,random_state,None


,id,desc,feature_name,value
0,1,Baseline model,intercept,0.389600
1,1,Baseline model,mean radius,2.142483
2,1,Baseline model,mean texture,0.073479
3,1,Baseline model,mean perimeter,-0.148649
4,1,Baseline model,mean area,0.015644
5,1,Baseline model,mean smoothness,-0.104537
6,1,Baseline model,mean compactness,-0.407278
7,1,Baseline model,mean concavity,-0.594602
8,1,Baseline model,mean concave points,-0.263310
9,1,Baseline model,mean symmetry,-0.155131


,id,desc,train_score,test_score
0,1,Baseline model,0.96044,0.938596


<a id='p2.2'></a>
## Problem 3 (10 points): 
Create a second model using only the features included in the list below (in `feature_cols`).  Insert the relevant arrays into the corresponding tables in the database.

Remember to update the `id` and `desc` values for the second iteration.

#### Suggestions
* Name this second model `"Reduced model"`.

In [11]:
feature_cols = ['mean radius',
                'texture error',
                'worst radius',
                'worst compactness',
                'worst concavity']

In [12]:
# grabbing the relevant columns
X_train_r = X_train[feature_cols]
X_test_r = X_test[feature_cols]

# creating a second model using reduced features
clf_r = LogisticRegression()
clf_r.fit(X_train_r, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
# saving reduced model data
save_model_data(model_id = 2, 
                model_desc = "Reduced model", 
                feature_names = feature_cols, 
                X_train = X_train_r, 
                X_test = X_test_r, 
                y_train = y_train, 
                y_test = y_test, 
                model = clf_r)

True

<a id='p2.3'></a>
## Problem 4 (10 points): 
Create one last model using an **l1-penalty** ($L_{1}$) term and **all** the features. Insert the relevant arrays into the corresponding tables in the database.

**Hint:** Refer to the `penalty` parameter of the `LogisticRegression` class.

#### Suggestions
Call this model `"L1 penalty model"`.

In [14]:
# changing the penalty parameter from default l2 to l1
clf_l1 = LogisticRegression(penalty = 'l1')
clf_l1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
# saving L1 penalty model data
save_model_data(model_id = 3, 
                model_desc = "L1 penalty model", 
                feature_names = data.feature_names, 
                X_train = X_train, 
                X_test = X_test, 
                y_train = y_train, 
                y_test = y_test, 
                model = clf_l1)

True

<a id='part_3'></a>

# Part 3:  Queries

<a id='p3.1'></a>
## Problem 5 (10 points): 
Query the database to identify the model with the highest validation score.
* Print the id of the best model and the corresponding validation score.
  ```bash
  Best model id: 
  Best validation score:
  ```
* Print the feature names and corresponding coefficients of that model.

In [16]:
# id of the best model and the corresponding validation score
query = '''SELECT id, MAX(test_score) FROM model_results'''
best_model_id, best_validation_score = cursor.execute(query).fetchall()[0]

print ('Best model id: {}'.format(best_model_id))
print ('Best validation score: {}'.format(best_validation_score))

Best model id: 3
Best validation score: 0.956140350877193


In [17]:
# feature names and corresponding coefficients of best model
query = '''SELECT B.feature_name, B.value FROM 
           (SELECT id, MAX(test_score) FROM model_results) as A LEFT JOIN model_coeffs as B
           ON A.id = B.id'''
best_params = cursor.execute(query).fetchall()
display(viz_tables(['feature_name', 'value'], query))

,feature_name,value
0,intercept,0.000000
1,mean radius,4.667530
2,mean texture,0.003580
3,mean perimeter,-0.317228
4,mean area,0.001765
5,mean smoothness,0.000000
6,mean compactness,0.000000
7,mean concavity,0.000000
8,mean concave points,0.000000
9,mean symmetry,0.000000


<a id='p3.2'></a>
## Problem 6 (10 points): 

Use the coefficients extracted in the previous question to reproduce the validation score (accuracy) of the best performing model (as stored in the database).

**Hint:** You should be able to achieve this by overwriting the relevant variables in the Logistic regression object, i.e. there is no need write your own formula to generate individual predictions (you are welcome to do this if you want).

#### Remarks
The problem demos a simple scenario in which someone with access to your database can easily reproduce your results.

In [18]:
# obtaining the intercept / coefficients and making into valid data type for model
best_intercept = np.array([best_params[0][1]])

best_coeffs = []

for i in range(1, len(best_params)):
    best_coeffs.append(best_params[i][1])
    
best_coeffs = np.array([best_coeffs])

In [19]:
# Using the old CLF from problem 2 and overwriting intercept / coefficients
clf.intercept_ = best_intercept
clf.coef_ = best_coeffs

In [20]:
clf.score(X_test, y_test)

0.956140350877193